# How do I _copy_ an app from one of my other _projects_?
## Overview
Apps include both tools (individual bioinformatics utilities) and workflows (chains or pipelines of connected tools). Apps are located in one of two places: the [Seven Bridges Public Apps repository](https://igor.sbgenomics.com/public/apps) or a user-created project.

## Objective
You can copy an app from the Public App repository or from a project of which you're a member. Here, we focus on **copying an app from another project**. [[reference](http://docs.sevenbridges.com/docs/copy-an-app)]

## Procedure
1. We'll define the project and app IDs
2. We'll copy an app to the destination project.

## Prerequisites
 1. You need to be a member (or owner) of at least two projects, one with the app that should be copied and another one where the app needs to be copied to. Learn more about creating a new project. [[recipe](projects_makeNew.ipynb)] [[reference](http://docs.sevenbridges.com/docs/create-a-new-project)].
 2. You need your **authentication token** and the API needs to know about it. See <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a> for details. Learn more about [obtaining your authentication token](http://docs.sevenbridges.com/v1.0/docs/get-your-authentication-token).
 3. You have at least one app in one project. If not, you can copy one from the Public Apps repository. [[recipe](apps_copyFromPublicApps.ipynb)] [[reference](http://docs.sevenbridges.com/docs/copy-an-app)]
 4. You can find app and project ID. We can find entity IDs via API or on the platform visual interface. [[reference](https://docs.sevenbridges.com/docs/the-api#identifying-projects-users-apps-files-tasks-and-inputs)]
 5. You understand how to list apps within a project. We will use this call below and find the copied app. [[recipe](apps_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-apps-available-to-you)]
 
Note that as with all okAPI recipes, this recipe makes use of the [Seven Bridges Public API Python library](../Tutorials/SBPLAT/Setup_API_environment.ipynb)
 
## Imports
We import the `Api` class from the official `sevenbridges-python` bindings below.  
We also import the `Conflict` error to handle situations when trying to copy over an existing app.

In [ ]:
import sevenbridges as sbg
from sevenbridges.errors import Conflict

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy an app you can access
First, we'll define which app we want to copy and to which project. Then, we'll try to copy an app to the second project. (Note that here you can also choose to copy a specific version of the app, by changing the last fragment of the `app_id` variable to the version you want).

The critical information for this `POST` request is the **app_id**. Note, you are **not** allowed to copy the same app **and** assign the same name<sup>1</sup> more than once. If you change the name, it is ok. 

To make these results very obvious, use an empty project as your my\_project (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or add the _name_ arguement to something ridiculous like 'Orange Mocha Frapachino Maker'. In this example, we will handle these situations with a predefined error `Conflict` for this situation.

<sup>1</sup> Note that setting the **name** of a copied app defines the app **id**, which must be unique on the platform. Saving over an existing ID will raise a `Conflict` error.

In [ ]:
# [USER INPUT] Set destination_project_id and app_id here:

# Set project ID in format 'user-name/project-name' or in enterprise mode 'division-name/project-name'
destination_project_id = 'rfranklin/destination-project-name'
# Set app ID in format 'user-name/project-name/app-name/revision' or in division context `division-name/project-name/app-name/revision'
# Omitting the revision will use the latest revision of the app
app_id = 'rfranklin/source-project-name/graf-germline-variant-detection-workflow-1-0/1' 

# Get the app and the project and try to copy the app
source_app = api.apps.get(app_id)
destination_project = api.projects.get(destination_project_id)
try:
    new_app = source_app.copy(project=destination_project_id)
    print('App {} copied to Project {}.'.format(source_app.name, destination_project.name))
except Conflict:
    new_app = [a for a in api.apps.query(project=destination_project) if a.name == source_app.name][0]
    print('App already exists in the destination project, reusing existing app.')

# re-list apps in target project to verify the copy worked
my_app_names = [a.name for a in api.apps.query(
    project=destination_project_id, limit=100
).all()]
    
if source_app.name in my_app_names:
    print('Sucessfully copied or reused one app!')
else:
    print('Something went wrong...')  

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-an-app).